In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Loading the files

In [4]:
# Let's use OS module to find the current directory and form a filepath
# which will be used to read files.
# We do this to avoid FileNotFound error so that the project runs smoothly on any system

# Let's get the current working directory
base_path = os.getcwd()

# Now let's form the file paths to all 3 of our datafiles
# Variable names are chosen to reflect the type of data present in the datafiles
articles_path = os.path.join(base_path, 'Group 5', 'articles.GSTM1.csv')
authors_path = os.path.join(base_path, 'Group 5', 'authors.GSTM1.csv')
paper_counts_path = os.path.join(base_path, 'Group 5', 'paper_counts.csv')

# Loading the files using pandas 
articles = pd.read_csv(articles_path)
authors = pd.read_csv(authors_path)
paper_counts = pd.read_csv(paper_counts_path)

## Displaying the files 

In [5]:
articles

,PMID,Title,Abstract,ISSN,Journal,Location,Year,FirstAuthorForename,FirstAuthorLastname,FirstAuthorInitials,FirstAuthorAffiliation
0,10548311,A systematic review of genetic polymorphisms a...,Studies investigating the relationship between...,1055-9965,"Cancer epidemiology, biomarkers & prevention :...",(8) 843-54,1999,A M,Dunning,AM,Cancer Research Campaign Human Cancer Genetics...
1,10548400,Prevalence of disease-related DNA polymorphism...,Genetic susceptibility polymorphisms may be of...,0959-8278,European journal of cancer prevention : the of...,(8) 441-7,1999,K,Woodson,K,"Division of Clinical Sciences, National Cancer..."
2,10564681,Glutathione S-transferase mu and theta polymor...,The enzymes encoded by the glutathione S-trans...,0027-8874,Journal of the National Cancer Institute,(91) 1960-4,1999,M,García-Closas,M,"M. García-Closas, Division of Cancer Epidemiol..."
3,10566550,Effect of vitamin intervention on the relation...,The GSTM1 (glutathione S-transferase mu-1) nul...,1055-9965,"Cancer epidemiology, biomarkers & prevention :...",(8) 965-70,1999,K,Woodson,K,"Cancer Prevention Studies Branch, Division of ..."
4,10571654,Genetic cancer susceptibility and DNA adducts:...,Preventive strategies require identification o...,0361-090X,Cancer detection and prevention,(23) 445-53,1999,H,Bartsch,H,Division of Toxicology and Cancer Risk Factors...
...,...,...,...,...,...,...,...,...,...,...,...
2733,37920524,A case-control study and systematic review of ...,GSTM1 deletion was reported to be associated w...,2405-8440,Heliyon,(9) e21183,2023,Jie,Peng,J,"School of Clinical Medicine, The First Affilia..."
2734,37948104,GST polymorphism as a predictive biomarker for...,What is the central question of this study? Ge...,1469-445X,Experimental physiology,10.1113/EP091339,2023,Harsh,Yadav,H,Department of Pulmonary Medicine Government Me...
2735,37993433,Distinct transcriptomic profiles in children p...,Although the genetic basis and pathogenesis of...,2041-1723,Nature communications,(14) 7630,2023,Jake,Lin,J,"Prostate Cancer Research Center, Faculty of Me..."
2736,38019237,Genetic Polymorphism in Xenobiotic Metabolisin...,Oxidative stress combined with nullity of xeno...,2476-762X,Asian Pacific journal of cancer prevention : A...,(24) 3795-3804,2023,Arun,Pandiyan,A,"ICMR-National Institute of Nutrition, Tarnaka,..."


In [6]:
authors

,PMID,AuthorN,AuthorForename,AuthorLastname,AuthorInitials,AuthorAffiliation
0,10548311,1,A M,Dunning,AM,Cancer Research Campaign Human Cancer Genetics...
1,10548311,2,C S,Healey,CS,NaN
2,10548311,3,P D,Pharoah,PD,NaN
3,10548311,4,M D,Teare,MD,NaN
4,10548311,5,B A,Ponder,BA,NaN
...,...,...,...,...,...,...
19275,38019237,7,Padmaja R,Jonnalagadda,PR,"ICMR-National Institute of Nutrition, Tarnaka,..."
19276,38073224,1,Masoud Hassanzadeh,Makoui,MH,Tehran University of Medical Sciences Faculty ...
19277,38073224,2,Shiva,Fekri,S,Zanjan University of Medical Sciences Faculty ...
19278,38073224,3,Reza Hassanzadeh,Makoui,RH,Zanjan University of Medical Sciences Faculty ...


In [7]:
paper_counts

,Year,Count
0,1799,1
1,1801,1
2,1802,1
3,1805,1
4,1866,1
...,...,...
138,2020,1132241
139,2021,1277048
140,2022,1324650
141,2023,1302390
